In [1]:
import time
from datetime import datetime

end_time = datetime.now().replace(hour=11, minute=3, second=0)
if datetime.now() > end_time:
    end_time = end_time.replace(day=end_time.day + 1)

while datetime.now() < end_time:
    time.sleep(60)

In [3]:
from trading_helper import TradingHelper
import time
import statistics
import math
from datetime import datetime as dt
app=TradingHelper()
mail="cirigliano.santiago@gmail.com"
cantidad_minima=5
tickers=["GD38","GD41"]
combinations=[tickers,[tickers[1],tickers[0]]]
lista_ratios=[]
while dt.now().hour < 11: time.sleep(60)
while 11 <= dt.now().hour < 17:
  if dt.now().hour >= 17:break
  for tickers in combinations:
    time.sleep(0.5)
    try:
        numerador = app.get_snapshot(mail=mail, ticker=tickers[0])
        denominador = app.get_snapshot(mail=mail, ticker=tickers[1])
    except Exception as e:
        # Reintentar mientras el error sea un 500
        while "Error code: 500" in str(e):
            try:
                numerador = app.get_snapshot(mail=mail, ticker=tickers[0])
                denominador = app.get_snapshot(mail=mail, ticker=tickers[1])
                break  # Salir del bucle si la solicitud es exitosa
            except Exception as e:
                if "Error code: 500" not in str(e):
                    print(f"Error al obtener snapshot: {e}")  # Registro del error diferente
                    break  # Salir del bucle si es otro error
                continue  # Continúa reintentando si es un error 500
    numerador =  [item for item in numerador if item['currency'] == 'ARS' and item['term'] == '24hs'][0]
    denominador =  [item for item in denominador if item['currency'] == 'ARS' and item['term'] == '24hs'][0]

    precio_venta_numerador=numerador['bids'][0]['price']#precio al que vendo este ticker
    vol_venta_numerador=numerador['bids'][0]['size']

    precio_compra_denominador=denominador['asks'][0]['price']#precio al que compro este ticker
    vol_compra_denominador=denominador['asks'][0]['size']

    #tengo numerador y lo vendo para comprar denominador
    ratio=precio_venta_numerador/precio_compra_denominador
    lista_ratios.append(ratio)

    if len(lista_ratios)>30: 
        de_numerador = statistics.stdev(lista_ratios)
        promedio_numerador=statistics.mean(lista_ratios)
        lista_ratios =lista_ratios[-30:]
        if promedio_numerador+de_numerador+de_numerador>ratio:
            if ratio > 1: 
                sell_quantity = cantidad_minima
                buy_quantity = math.floor(cantidad_minima*ratio)
            elif ratio < 1:
                sell_quantity = math.floor(cantidad_minima*ratio)
                buy_quantity = cantidad_minima
            elif ratio == 1:
                sell_quantity = cantidad_minima
                buy_quantity = cantidad_minima
            sell_order=app.place_sell_order(mail=mail,
                                            ticker=tickers[0],
                                            quantity=sell_quantity,
                                            price=precio_venta_numerador,
                                            term="24hs",
                                            currency="pesos")
            print(f"Orden de venta {tickers[0]} {sell_order}")
            order_status=app.check_order_status(mail=mail,
                                                order_number=sell_order["Orden"])
            print(order_status)
            start_time = time.time()
            while order_status == "MARKET":
                elapsed_time = time.time() - start_time
                if elapsed_time > 4:
                    cancel_order=app.cancel_order(mail=mail,order_number=sell_order["Orden"])
                    print(f"Orden cancelada por Timeout: {cancel_order}")
                    break
                order_status=app.check_order_status(mail=mail,
                                                order_number=sell_order["Orden"])
                print(f"Order Status de venta {order_status}")
                
            if order_status == "EXECUTED":
                buy_order=app.place_buy_order(mail=mail,
                                            ticker=tickers[1],
                                            quantity=buy_quantity,
                                            price=precio_compra_denominador,
                                            term="24hs",
                                            currency="pesos")
                print(f"Orden de compra {tickers[1]} {buy_order}")
                order_status=app.check_order_status(mail=mail,
                                                    order_number=buy_order["Orden"])
                while order_status == "MARKET":
                    try:
                        time.sleep(1)
                        order_status=app.check_order_status(mail=mail, order_number=buy_order["Orden"])
                        if order_status =="FILLED": break
                        cancel_order=app.cancel_order(mail=mail,order_number=buy_order["Orden"])
                        precio_compra_denominador = precio_compra_denominador - 0.1
                        buy_order=app.place_buy_order(mail=mail,
                            ticker=tickers[1],
                            quantity=buy_quantity,
                            price=precio_compra_denominador,
                            term="24hs",
                            currency="pesos")
                    except Exception as e: print(e)
                    
                print(f"Order status de Compra {order_status}")
                if buy_order["Success"] == True: print("Negocio ATR")

In [ ]:
tickers=["GD38","GD41"]
print([tickers,[tickers[1],tickers[0]]])

[['GD38', 'GD41'], ['GD41', 'GD38']]
